In [489]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.stats.multitest as smm

### Simulated data

Here we test a simulated hypothesis based on the data drown from standard normal distribution.

<b>H0</b>: value of x is not different from 0

<b>H1</b>: value of x is larger than 0

The first 900 observations should fail to reject the null hypothesis: they are, in fact, drawn from a standard normal distribution and any 2 difference between the observed value and 0 is just due to chance. 

The last 100 observations should reject the null hypothesis: the difference between these values and 0 is not due to
chance alone.

In [490]:
numtests = 1000
alpha = 0.05

np.random.seed(42)

In [491]:
d1 = np.random.normal(0, 1, 900)
d2 = np.random.normal(3, 1, 100)
data1 = np.concatenate((d1, d2), axis=0)
print(data1[:10])

[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
  1.57921282  0.76743473 -0.46947439  0.54256004]


In [492]:
d3 = st.norm.rvs(0, 1, size=900)
d4 = st.norm.rvs(2, 1, size=100)
data2 = np.concatenate((d3, d4), axis=0)

In [493]:
prob = st.norm.pdf(data1)
print(prob[:10])

[ 0.35264231  0.39514715  0.32345711  0.12508666  0.38815426  0.38815575
  0.11464727  0.29718021  0.35731354  0.34434052]


In [494]:
test = prob > alpha
print(test[:100])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [495]:
print("True: %d" % sum([t for t in test[:900]]))
print("False: %d" % sum([not t for t in test[:900]]))

True: 865
False: 35


In [496]:
print("True: %d" % sum([t for t in test[900:]]))
print("False: %d" % sum([not t for t in test[900:]]))

True: 18
False: 82


In [497]:
print("Type I error rate: %.4f" % (sum([not t for t in test[:900]]) / 900.))
print("Type II error rate: %.4f" % (sum([t for t in test[900:]]) / 100.))

Type I error rate: 0.0389
Type II error rate: 0.1800


### Bonferroni correction

In [498]:
bonf_test = prob > alpha/numtests

In [499]:
# from statsmodels.stats.multitest
# rej, pval_corr = smm.multipletests(prob, alpha=0.05, method='b')[:2]

In [500]:
print("True: %d" % sum([t for t in bonf_test[:900]]))
print("False: %d" % sum([not t for t in bonf_test[:900]]))

print("True: %d" % sum([t for t in bonf_test[900:]]))
print("False: %d" % sum([not t for t in bonf_test[900:]]))

True: 900
False: 0
True: 88
False: 12


In [517]:
print("Type I error rate: %.4f" % (sum([not t for t in bonf_test[:900]]) / 900.))
print("Type II error rate: %.4f" % (sum([t for t in bonf_test[900:]]) / 100.))

Type I error rate: 0.0000
Type II error rate: 0.8800


### False discovery rate

In [520]:
prob_sorted = np.sort(prob)

fdrtest = None
for i in range(numtests):
    #print(next((i for i,x in enumerate(prob_sorted) if x == prob[i]), None))
    position_value = prob[i] > next(i for i,x in enumerate(prob_sorted) if x == prob[i]) * alpha/numtests
    fdrtest = np.append(fdrtest, position_value)

In [521]:
print("True: %d" % sum([t for t in fdrtest[1:900]]))
print("False: %d" % sum([not t for t in fdrtest[1:900]]))

print("True: %d" % sum([t for t in fdrtest[900:]]))
print("False: %d" % sum([not t for t in fdrtest[900:]]))

True: 896
False: 3
True: 51
False: 50


In [522]:
print("Type I error rate: %.4f" % (sum([not t for t in fdrtest[:900]]) / 900.))
print("Type II error rate: %.4f" % (sum([t for t in fdrtest[900:]]) / 100.))

Type I error rate: 0.0044
Type II error rate: 0.5100


### Benjamini/Hochberg (non-negative)

In [523]:
rej, pval_corr = smm.multipletests(prob, alpha=0.05, method='fdr_i')[:2]

In [524]:
bh_nn_test = pval_corr > 0.05

In [525]:
print("True: %d" % sum([t for t in bh_nn_test[:900]]))
print("False: %d" % sum([not t for t in bh_nn_test[:900]]))

print("True: %d" % sum([t for t in bh_nn_test[900:]]))
print("False: %d" % sum([not t for t in bh_nn_test[900:]]))

True: 898
False: 2
True: 52
False: 48


In [526]:
# two staged
rej, pval_corr = smm.multipletests(prob, alpha=0.05, method='fdr_tsbh')[:2]

In [527]:
bh_nn_test_2 = pval_corr > 0.05

In [528]:
print("True: %d" % sum([t for t in bh_nn_test_2[:900]]))
print("False: %d" % sum([not t for t in bh_nn_test_2[:900]]))

print("True: %d" % sum([t for t in bh_nn_test_2[900:]]))
print("False: %d" % sum([not t for t in bh_nn_test_2[900:]]))

True: 898
False: 2
True: 52
False: 48


### Benjamini/Yekutieli (negative)

In [529]:
rej, pval_corr = smm.multipletests(prob, alpha=0.05, method='fdr_n')[:2]

In [530]:
by_n_test = pval_corr > 0.05

In [531]:
print("True: %d" % sum([t for t in test[:900]]))
print("False: %d" % sum([not t for t in test[:900]]))

print("True: %d" % sum([t for t in test[900:]]))
print("False: %d" % sum([not t for t in test[900:]]))

True: 865
False: 35
True: 18
False: 82


### Benjamini/Krieger/Yekutieli

In [532]:
rej, pval_corr = smm.multipletests(prob, alpha=0.05, method='fdr_tsbky')[:2]

In [533]:
bky_test = pval_corr > 0.05

In [534]:
print("True: %d" % sum([t for t in bky_test[:900]]))
print("False: %d" % sum([not t for t in bky_test[:900]]))

print("True: %d" % sum([t for t in bky_test[900:]]))
print("False: %d" % sum([not t for t in bky_test[900:]]))

True: 898
False: 2
True: 52
False: 48


### QA and Summary

<b>1. Should we correct primary and secondary goals separately?</b>

There are two possibilities for the correction:
1. Across all KPI and subgroups
2. Correction per KPI

Currently, the second approach seems to be more reasonable:
- each KPI is a separate hypothesis (many of KPIs are independent of each other and don't share the same effect (good if uplift, good if down-lift))
- significance is decided within each KPI, so that the more subgroups you have in terms of one KPI there will be more likelihood to get at least one significant result, which is the reason for correction.
- for the final outcome we define recommendation rules (decides the resulting acceptance criteria across primary and secondary KPIs)


First approach makes sense if KPIs are positively correlate with each other, share same effect and whether we want to replace the acceptance criteria.
One of the drawbacks is that correction across all kpi-groups will adjust α to be very small. It will be impossible to remove one of the KPI from the decision.

<b>2. Which correction should we use and why?</b>

<b>Bonferroni correction</b>:
+ it sets the significance cut-off at α/n, where n is the number of experiments.
+ simplest and straigtforward correction.
- it may be very conservative (assumes all tests are independent of each other), which may lead to a high rate of false negatives.
- is not applicable if there is a positive correlation between tests.

<b>The False Discovery Rate</b>:
is the proportion of false positives among all significant results.
The FDR works by estimating some rejection region so that, on average, FDR < α.


